# задание:
- 1. Взять модель для суммаризации текстов https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta и запустить суммаризацию текстов на тестовой части вот этого датасета: https://huggingface.co/datasets/IlyaGusev/gazeta. Код, который это делает, есть в карточке модели. Посчитать метрики blue и rouge
- 2. Дообучить модель google/mt5-small для суммаризации текстов из датасета https://huggingface.co/datasets/IlyaGusev/gazeta, запустить суммаризацию на тестовой части датасета, посчитать метрики blue и rouge
Таким образом, у вас должно получиться сравнение метрик для оценки качества суммаризации, сделанной двумя разными моделями

# Установка необходимых библиотек

In [1]:
!pip install datasets
!pip install torch torchvision torchaudio
!pip install transformers
!pip install rouge-score nltk
!pip install sentencepiece
!pip install evaluate

  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.17.3 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install transformers[torch] -U

  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.17.3 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Импорты

In [3]:
import nltk
nltk.download('punkt')
from datasets import load_dataset, load_metric
from transformers import (
    DataCollatorForSeq2Seq,
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    pipeline,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
) 
import evaluate
import numpy as np
import datasets

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emely\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Загрузка предварительно обученного токенизатора и модели (MT5)

In [4]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# Параметры

In [10]:
# Установка максимального количества токенов для исходных и целевых последовательностей
max_target_tokens_count = 20
max_source_tokens_count = 1024

# Загрузка метрик Rouge и Bleu для оценки
rouge = load_metric("rouge")
bleu = evaluate.load("bleu")

# Функции

In [11]:
# Определение функции предобработки для датасета
def preprocess_function(examples):
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=max_source_tokens_count, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_tokens_count, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Определение функции для вычисления метрик во время оценки
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = {}
    result_rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result.update({key: value.mid.fmeasure * 100 for key, value in result_rouge.items()})
    result_bleu = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result["bleu"] = result_bleu["bleu"] * 100
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])
    return {k: round(v, 4) for k, v in result.items()}

In [12]:
# Загрузка датасета Gazeta и применение функции предобработки
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0")
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Инициализация коллатора данных для задачи Seq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Моделька

In [12]:
!pip uninstall accelerate -y
!pip uninstall transformers -y
!pip install accelerate==0.20.1
!pip install transformers[torch] -U

Found existing installation: accelerate 0.20.1
Uninstalling accelerate-0.20.1:
  Successfully uninstalled accelerate-0.20.1
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
  Using cached accelerate-0.20.1-py3-none-any.whl (227 kB)


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.1
    Uninstalling accelerate-0.20.1:
      Successfully uninstalled accelerate-0.20.1


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
# Определение аргументов для тренировки модели Seq2Seq
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=25,
    logging_steps=25,
    learning_rate=4e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=False,
    predict_with_generate=True,
    generation_max_length=max_target_tokens_count,
    generation_num_beams=5
)

In [17]:
# Инициализация Seq2SeqTrainer с определенной моделью, аргументами тренировки и датасетами
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Тренировка модели

In [18]:
trainer.train()

  0%|          | 0/238 [00:00<?, ?it/s]

{'loss': 3.0563, 'learning_rate': 0.00035798319327731096, 'epoch': 0.1}


  0%|          | 0/1699 [00:00<?, ?it/s]

{'eval_loss': 2.4588754177093506, 'eval_rouge1': 6.5166, 'eval_rouge2': 1.4112, 'eval_rougeL': 6.4723, 'eval_rougeLsum': 6.4584, 'eval_bleu': 4.5061, 'eval_gen_len': 18.9619, 'eval_char_len': 69.9463, 'eval_runtime': 1557.0421, 'eval_samples_per_second': 4.363, 'eval_steps_per_second': 1.091, 'epoch': 0.1}
{'loss': 2.7557, 'learning_rate': 0.00031596638655462185, 'epoch': 0.21}


  0%|          | 0/1699 [00:00<?, ?it/s]

...

  0%|          | 0/1699 [00:00<?, ?it/s]

{'eval_loss': 2.228257894515991, 'eval_rouge1': 6.998, 'eval_rouge2': 1.6984, 'eval_rougeL': 6.959, 'eval_rougeLsum': 6.9597, 'eval_bleu': 5.1361, 'eval_gen_len': 18.9903, 'eval_char_len': 69.3083, 'eval_runtime': 1524.013, 'eval_samples_per_second': 4.457, 'eval_steps_per_second': 1.115, 'epoch': 0.84}
{'loss': 2.6484, 'learning_rate': 2.184873949579832e-05, 'epoch': 0.94}


  0%|          | 0/1699 [00:00<?, ?it/s]

{'eval_loss': 2.2456958293914795, 'eval_rouge1': 7.0569, 'eval_rouge2': 1.7741, 'eval_rougeL': 6.9995, 'eval_rougeLsum': 6.9869, 'eval_bleu': 5.0972, 'eval_gen_len': 18.9941, 'eval_char_len': 69.4322, 'eval_runtime': 1522.497, 'eval_samples_per_second': 4.462, 'eval_steps_per_second': 1.116, 'epoch': 0.94}
{'train_runtime': 27582.2646, 'train_samples_per_second': 2.21, 'train_steps_per_second': 0.009, 'train_loss': 2.7541401967281054, 'epoch': 1.0}


TrainOutput(global_step=238, training_loss=2.7541401967281054, metrics={'train_runtime': 27582.2646, 'train_samples_per_second': 2.21, 'train_steps_per_second': 0.009, 'train_loss': 2.7541401967281054, 'epoch': 1.0})

# Оценка обученной модели

In [19]:
trainer.evaluate()

  0%|          | 0/1699 [00:00<?, ?it/s]

{'eval_loss': 2.229231595993042,
 'eval_rouge1': 7.0972,
 'eval_rouge2': 1.781,
 'eval_rougeL': 7.0354,
 'eval_rougeLsum': 7.0324,
 'eval_bleu': 5.069,
 'eval_gen_len': 18.9885,
 'eval_char_len': 69.3579,
 'eval_runtime': 7410.3835,
 'eval_samples_per_second': 0.917,
 'eval_steps_per_second': 0.229,
 'epoch': 1.0}